In [ ]:
import re

import parse
import pdfplumber
import pandas as pd
from collections import namedtuple

pd.set_option('display.max_rows', None)  # or 1000


import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 

In [ ]:
import os

# Mudando o diretório para H:/My Drive
os.chdir('H:/Meu Drive/TRABALHOS CDT/CDT_2023_FINAL/')

In [ ]:
import os
################## MUDAR O ANO AQUI ####################

ano = '2_ANO'
classes = []

diretorio = 'H:/Meu Drive/TRABALHOS CDT/CDT_2023_FINAL/{ano}'.format(ano = ano)

# Lista apenas os arquivos no diretório

arquivos = [arquivo for arquivo in os.listdir(diretorio) if os.path.isfile(os.path.join(diretorio, arquivo))]

# Imprime o nome de cada arquivo
for arquivo in arquivos:
    classes.append(arquivo)

display(classes)

In [ ]:
name_re = re.compile(r'(Nome:) (.*) (Data)')
ra_re = re.compile(r'(.*) (RA:) (.*) (RM:)')
#turma_re = re.compile(r'(Turma:) (.*) (ANUAL)')

In [ ]:
   
grades = {}
names = {}
class_ = {}

for c in classes[0:]:
    
    turma = c

    files = ['{directory}/{classes}'.format(directory = ano, classes=c)]

    for file in files:
        with pdfplumber.open(file) as pdf:

            for page in pdf.pages[0:]:

                    tables = page.find_tables()
                    
                    text = page.extract_text()
                    
 ################### ARMAZENANDO RA E NOME ##################                   
                    
                    for line in text.split('\n'):
                        name = name_re.search(line)
                        
                        if name:
                            name_storage = name.group(2)
                            
                        ra = ra_re.search(line)
                        
                        if ra:
                            ra_storage = str(ra.group(3))
                            
                            
                        
##################### ARMAZENANDO AS NOTAS ######################
                    
                    for tables_num in range (0,(len(tables))):
                        
                        table = tables[tables_num].extract(x_tolerance = 5)
                    


                        #tabela crua = TC
                        tc = pd.DataFrame(table, columns=table[0])
                    
                        #tabela com notas das materias = TM
                        try:
                            tc_t = tc.transpose()
                            tm = tc_t.iloc[1:, 10:14] \
                                .replace({'ET': None
                                          , 'ES': None
                                          , 'EP': None
                                          , '-':None
                                          , '': None})
                            
                            columnNames = ['Materia', 'BIM_1','BIM_2', 'BIM_3', 'BIM_4']
                            #tm.columns = columnNames
                            
                            
                            #resetando para incluir a coluna "index" no texto
                            tm.reset_index( inplace=True)
                            tm = tm.applymap(lambda x: x.replace('\n', '') if isinstance(x, str) else x)
                            
                            # Dicionário de mapeamento de colunas antigas para colunas novas
                            mapeamento_colunas = {'index':'Materia', 10: 'BIM_1', 11: 'BIM_2', 12: 'BIM_3', 13: 'BIM_4'}

                            # Use o método rename para renomear as colunas
                            tm.rename(columns=mapeamento_colunas, inplace=True)
                                
                            tm = tm. \
                                    drop(index=tm[tm['Materia'].isin(['LEITURA E CONVERSAÇÃO EM ESPANHOL'
                                                                     , 'SOCIOLOGIA'
                                                                     ])].index)
                            
########################### ADICIONANDO TABELA DE NOTAS A UM RA ###################################
                            
        
                            
                            if ra_storage not in grades.keys() and len(tm.columns) >= 5:
                                grades[ra_storage] = tm
                                names[ra_storage] = name_storage
                                
                            elif ra_storage in grades.keys() and len(tm.columns) >= 5:
                                grades[ra_storage] = pd.concat([grades[ra_storage], tm], ignore_index=True)                                                          
                                
                            
                                    
                            if ra_storage not in class_ and len(tm.columns) >= 5:
                                class_[ra_storage] = []
                                class_[ra_storage].append(c)

                            else:
                                if c in class_[ra_storage]:
                                    True
                                else:
                                    class_[ra_storage].append(c)

                            
                        except KeyError:
                            True      



In [ ]:
import pandas as pd

#função para consolidar máterias repetidas, pegando sempre a maior nota de cada aluno para um dado bimestre e matéria

def consolidacao(all_subjects): 

    df = pd.DataFrame(all_subjects)

    df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

    valores_maximos_por_materia = df.groupby('Materia').agg({'BIM_1': 'max', 'BIM_2': 'max', 'BIM_3': 'max', 'BIM_4': 'max'})
    
    return valores_maximos_por_materia




In [ ]:
#Nas células anteriores, o código lista TODAS as matérias por linha, com as colunas contendo as notas por bimestre.
#Alguns alunos migraram de turmas (para outros turnos ou para outras salas), por isso um mesmo aluno pode ter mais de
#uma nota para mesma matéria e bimestre. Para contornar isso, todas as matérias são listadas e depois agregadas, usando
#funcao consolidacao. Ela permite pegar a maior nota de cada aluno.


for key in names.keys():
    grades[key] = consolidacao(grades[key])
    

In [ ]:
#VALIDANDO AS MATÉRIAS QUE ESTÃO SENDO CONSIDERADAS
s = []
t = []

for key in names.keys():
    
    r = grades[key].iloc[:, :0]
    
    
    s.append(r.index.tolist())

for x in s:
    for p in x:
        t.append(p)
display(set(t))

In [ ]:
def medias_finais(df):

    df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
    numerico = df
    media_bimestre  = numerico.mean(axis=0)
    total_materias_bimestre = numerico.count(axis=0)
    
    produto = media_bimestre*total_materias_bimestre
    
    media_final = (produto.sum())/(total_materias_bimestre[0:].sum())


    return media_final
    

In [ ]:
x = 0

all_info_table = []

all_info_table = pd.DataFrame(columns=['Nome', 'RA', 'Turma', 'Media'])

for key in names.keys():
    
    aluno = {'Nome': names[key], 'RA': key, 'Media': medias_finais(grades[key]), 'Turma': class_[key]}
    all_info_table = all_info_table.append(aluno, ignore_index=True)


display(all_info_table)    
    
    
    

In [ ]:
all_info_table.to_csv('medias_cdt_2023_FINAL_{ano}.csv'.format(ano=ano), sep=';', index=False, decimal=',')


In [ ]:
########## PARA VALIDAR SÓ ADICIONAR O RA #############

ra = 'ra_do_aluno'
display(grades[ra].applymap(lambda x: str(x).replace('.', ',')).applymap(lambda x: str(x).replace('nan', '')))
display(all_info_table[all_info_table['RA'] == ra])
#display(medias_finais(grades['000113968949-6/SP']))